In [4]:
from gym_tictactoe.env import TicTacToeEnv
import numpy as np

env = TicTacToeEnv(size=5)

print("Available Actions:", env.available_actions())

actions = [0, 5, 1, 6, 2, 7, 3, 8, 4, 9]
for action in actions:
    observation, reward, done, trunc, _ = env.step(np.float32(action))
    print(observation, reward, done)
    env.render()
    print(done, reward)
    if done or trunc:
        print("Game Over!")
        break

print("Available Actions:", env.available_actions())


Available Actions: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 0 False
  O| | | | 
  ----------
   | | | | 
  ----------
   | | | | 
  ----------
   | | | | 
  ----------
   | | | | 
False 0
[1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 0 False
  O| | | | 
  ----------
  X| | | | 
  ----------
   | | | | 
  ----------
   | | | | 
  ----------
   | | | | 
False 0
[2. 2. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 0 False
  O|O| | | 
  ----------
  X| | | | 
  ----------
   | | | | 
  ----------
   | | | | 
  ----------
   | | | | 
False 0
[1. 1. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.] 0 False
  O|O| | | 
  ----------
  X|X| | | 
  ----------
   | | | | 
  ----------
   | | | | 
  ----------
   | | | | 
False 0
[2. 2. 2. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/home/frozenwolf/miniconda3/lib/python3.10/site-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


In [5]:
import gymnasium as gym

from stable_baselines3 import DQN, DDPG, PPO
from gym_tictactoe.env import TicTacToeEnv
from stable_baselines3.common.logger import configure
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from gym_tictactoe.env import TicTacToeEnv
import numpy as np

# env = TicTacToeEnv(size=5)
import numpy as np
tmp_path = "logs/"
# set up logger

env = TicTacToeEnv()
eval_callback = EvalCallback(env, best_model_save_path="ddpgmodel/",
                             log_path="logs/", eval_freq=1000,
                             deterministic=True, render=False, n_eval_episodes=10)
# n_actions = env.action_space.shape[-1]

action_noise = np.asarray(1)
action_noise = NormalActionNoise(mean=np.zeros(1), sigma=action_noise)

model = DDPG("MlpPolicy", env, verbose=1,  batch_size=200, action_noise=action_noise)
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])
model.set_logger(new_logger)
model.learn(total_timesteps=20000, log_interval=10, progress_bar=True,)
model.save("ddpgmodel")

vec_env = model.get_env()

del model # remove to demonstrate saving and loading

model = DDPG.load("ddpgmodel/best_model.zip")

obs, info = env.reset()


done = False
obs, info = env.reset()
observation = obs
while not done:
    action, _states = model.predict(observation, deterministic=True)
    observation, reward, done, trunc, _ = env.step(action)
    print(observation, reward, done)
    env.render()

    if done or trunc:
        print("Game Over!")
        break

# while True:
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     if terminated or truncated:
#         obs, info = env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/


Output()

/home/frozenwolf/miniconda3/lib/python3.10/site-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting 
input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.7      |
|    ep_rew_mean     | 0.6      |
| time/              |          |
|    episodes        | 10       |
|    fps             | 973      |
|    time_elapsed    | 0        |
|    total_timesteps | 77       |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.8      |
|    ep_rew_mean     | 0.45     |
| time/              |          |
|    episodes        | 20       |
|    fps             | 126      |
|    time_elapsed    | 1        |
|    total_timesteps | 156      |
| train/             |          |
|    actor_loss      | -0.167   |
|    critic_loss     | 0.03     |
|    learning_rate   | 0.001    |
|    n_updates       | 56       |
---------------------------------


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.64     |
|    ep_rew_mean     | 0.31     |
| time/              |          |
|    episodes        | 160      |
|    fps             | 59       |
|    time_elapsed    | 20       |
|    total_timesteps | 1224     |
| train/             |          |
|    actor_loss      | -0.729   |
|    critic_loss     | 0.0153   |
|    learning_rate   | 0.001    |
|    n_updates       | 1121     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.73     |
|    ep_rew_mean     | 0.28     |
| time/              |          |
|    episodes        | 170      |
|    fps             | 59       |
|    time_elapsed    | 21       |
|    total_timesteps | 1303     |
| train/             |          |
|    actor_loss      | -0.773   |
|    critic_loss     | 0.0189   |
|    learning_rate   | 0.001    |
|    n_updates       | 1200     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.78     |
|    ep_rew_mean     | 0.25     |
| time/              |          |
|    episodes        | 180      |
|    fps             | 59       |
|    time_elapsed    | 23       |
|    total_timesteps | 1382     |
| train/             |          |
|    actor_loss      | -0.786   |
|    critic_loss     | 0.0151   |
|    learning_rate   | 0.001    |
|    n_updates       | 1280     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 7.77     |
|    ep_rew_mean     | 0.25     |
| time/              |          |
|    episodes        | 190      |
|    fps             | 59       |
|    time_elapsed    | 24       |
|    total_timesteps | 1458     |
| train/             |          |
|    actor_loss      | -0.778   |
|    critic_loss     | 0.0257   |
|    learning_rate   | 0.001    |
|    n_updates       | 1355     |
---------------------------------


KeyboardInterrupt: 

In [3]:
env.action_space

Box(0.0, 8.0, (), float32)

In [2]:
env.action_space

Discrete(9)

In [14]:
import numpy as np
model = DDPG.load("ddpgmodel/best_model.zip")


done = False
obs, info = env.reset()

altter = True
observation = obs
while not done:
    print(observation)
    if altter:
        action = np.asarray(int(input("Enter action: ")))
        
    else:
        action, _states = model.predict(observation, deterministic=True)
    observation, reward, done, trunc, _ = env.step(action)
    altter = not altter
    # print(observation, reward, done)
    print(reward, done, trunc)
    env.render()

    if done or trunc:
        print("Game Over!")
        break

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


0 False False
   |O| 
  ------
   | | 
  ------
   | | 
[0. 2. 0. 0. 0. 0. 0. 0. 0.]
0 False False
   |O| 
  ------
   | | 
  ------
   | |X
[0. 1. 0. 0. 0. 0. 0. 0. 2.]
0 False False
   |O| 
  ------
  O| | 
  ------
   | |X
[0. 2. 0. 2. 0. 0. 0. 0. 1.]
0 False False
  X|O| 
  ------
  O| | 
  ------
   | |X
[2. 1. 0. 1. 0. 0. 0. 0. 2.]
0 False False
  X|O| 
  ------
  O|O| 
  ------
   | |X
[1. 2. 0. 2. 2. 0. 0. 0. 1.]
0 False False
  X|O| 
  ------
  O|O| 
  ------
   |X|X
[2. 1. 0. 1. 1. 0. 0. 2. 2.]
0 False False
  X|O| 
  ------
  O|O| 
  ------
  O|X|X
[1. 2. 0. 2. 2. 0. 2. 1. 1.]
0 False False
  X|O|X
  ------
  O|O| 
  ------
  O|X|X
[2. 1. 2. 1. 1. 0. 1. 2. 2.]


ValueError: invalid literal for int() with base 10: ''